In [ ]:
# 기본적인 Naive RAG -> 조건부엣지 검사 -> 웹 검색 -> 쿼리 재작성
import os
from dotenv import load_dotenv
from typing import Annotated, TypedDict, List
from langgraph.graph.message import add_messages
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver



load_dotenv()

True

In [2]:
# 랭그래프에서는 Retriever 와 Chain 을 따로 생성해서 다른 노드에 위치시킨다
from rag.pdf import PDFRetrievalChain

pdf = PDFRetrievalChain(["SPRI_AI_Brief_2023년12월호_F.pdf"]).create_chain()

pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

In [3]:
search_result = pdf_retriever.invoke("엔스로픽에 투자한 기업과 투자금액을 알려주세요.")
search_result

[Document(id='31fcd1d3-6e73-48fb-b015-eda97702a03b', metadata={'source': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 13, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화\nKEY Contents\nn 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은\n구글과 클라우드 서비스 사용 계약도 체결\nn 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인\n앤스로픽 및 오픈AI와 협력을 확대하는 추세\n£구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공'),
 Document(id='edabef32-92b8-4bda-b165-791b1c929bd0', metadata={'source': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 13, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "

In [4]:
answer = pdf_chain.invoke(
    {
        "question":"엔스로픽에 투자한 기업과 투자금액을 알려주세요.",
        "context":search_result,
        "chat_history":[],
    }
)
print(answer)

- Google — 최대 $2,000,000,000 투자 합의; 이 중 $500,000,000를 우선 투자, 향후 $1,500,000,000 추가 투자 예정. 2023년 2월에 Anthropic에 이미 $550,000,000 투자.
- Amazon — 최대 $4,000,000,000 투자 계획 발표(2023년 9월).

Source
- SPRI_AI_Brief_2023년12월호_F.pdf (page 13)


In [ ]:
# State 정의
from typing import Annotated, TypedDict, List
from langgraph.graph.message import add_messages

# 노드에서 다른 노드로 전달할 때 알려줄 State (상태결과)
class State(TypedDict):
    question: Annotated[str, "Question"]        # 질문
    context: Annotated[str, "Context"]          # 문서 검색 결과
    answer: Annotated[str, "Answer"]            # 답변
    messages: Annotated[list, add_messages]         # 메시지 누적 list

In [6]:
# Node 정의 (리트리브 노드, LLM 노드)
from langchain_teddynote.messages import messages_to_history
from rag.utils import format_docs

# 문서 검색 노드
def retrieve_document(state : State) -> State:
    # 마지막 질문
    latest_question = state["question"]
    # 문서에서 유사도 검색
    retrieved_docs = pdf_retriever.invoke(latest_question)
    retrieved_docs = format_docs(retrieved_docs)
    return State(context=retrieved_docs)

# LLM 답변 노드
def llm_answer(state : State)->State:
    # 질문을 상태에서 가져온다
    latest_question = state["question"]
    # 검색된 문서를 상태에서 가져옵니다
    context = state["context"]
    #체인을 호출하여 답변을 생성한다
    response = pdf_chain.invoke({
        "question" : latest_question,
        "context": context,
        "chat_history":messages_to_history(state["messages"])
    })
    return State(answer=response, messages=[("user", latest_question), ("as sistant", response)])


In [ ]:
# Edge 로 노드연결
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

